## Model Learning
- bagging 방법을 이용한 Random Forest
- Boosting 방법을 이용

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

import pandas as pd
import numpy as np

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = (8, 6)

In [2]:
header_list = ["age","workclass","fnlwgt","education","education_num","marital-status","occupation",
               "relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country",
               "income"] #15 columns (14 features, 1 target).
dataset = pd.read_csv("C:/Users/ksmin/ds_teamproject/Dataset/Adult Data.csv", names=header_list)

In [3]:
#y값을 변형하는 과정
dataset['income'] = dataset['income'].map(lambda x: 1 if x==' >50K' else 0)

In [4]:
# 특성과 타겟 분리
X = dataset.drop('income', axis=1)
y = dataset['income']

#LabelEncoder를 진행. 가중치가 없는 것으로 이용
String_index = []
for i, col in enumerate (X.columns):
    if X[col].dtype==object:
        String_index.append(col)

encoder = LabelEncoder()
for col in String_index:
    X[col] = encoder.fit_transform(X[col])

print(X.head())

   age  workclass  fnlwgt  education  education_num  marital-status  \
0   39          7   77516          9             13               4   
1   50          6   83311          9             13               2   
2   38          4  215646         11              9               0   
3   53          4  234721          1              7               2   
4   28          4  338409          9             13               2   

   occupation  relationship  race  sex  capital-gain  capital-loss  \
0           1             1     4    1          2174             0   
1           4             0     4    1             0             0   
2           6             1     4    1             0             0   
3           6             0     2    1             0             0   
4          10             5     2    0             0             0   

   hours-per-week  native-country  
0              40              39  
1              13              39  
2              40              39  
3       

In [5]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=dataset.income)

# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
#train 데이터를 살펴보자
train, test = train_test_split(dataset, test_size=0.2, random_state=2024)

print(train.shape)

(26048, 15)


In [7]:
# BaggingClassifier 모델 학습
bagging_model = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)
bagging_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_bagging = bagging_model.predict(X_test)
print("Bagging Classifier Accuracy:", accuracy_score(y_test, y_pred_bagging))
print("Bagging Classifier Classification Report:\n", classification_report(y_test, y_pred_bagging))

Bagging Classifier Accuracy: 0.855366190695532
Bagging Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91      4945
           1       0.73      0.63      0.68      1568

    accuracy                           0.86      6513
   macro avg       0.81      0.78      0.79      6513
weighted avg       0.85      0.86      0.85      6513



In [8]:
# AdaBoostClassifier 모델 학습
boosting_model = AdaBoostClassifier(
    DecisionTreeClassifier(),
    n_estimators=100,
    random_state=42
)
boosting_model.fit(X_train, y_train)

# 예측 및 평가
y_pred_boosting = boosting_model.predict(X_test)
print("AdaBoost Classifier Accuracy:", accuracy_score(y_test, y_pred_boosting))
print("AdaBoost Classifier Classification Report:\n", classification_report(y_test, y_pred_boosting))

c:\Users\ksmin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Classifier Accuracy: 0.8132964839551666
AdaBoost Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.88      4945
           1       0.61      0.63      0.62      1568

    accuracy                           0.81      6513
   macro avg       0.74      0.75      0.75      6513
weighted avg       0.82      0.81      0.81      6513



In [9]:
# 랜덤포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 예측
y_pred = rf_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8593582066635959
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4945
           1       0.74      0.64      0.69      1568

    accuracy                           0.86      6513
   macro avg       0.82      0.78      0.80      6513
weighted avg       0.85      0.86      0.86      6513



In [10]:
# XGBoost 모델 학습
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8747121142330723
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4945
           1       0.78      0.66      0.72      1568

    accuracy                           0.87      6513
   macro avg       0.84      0.80      0.82      6513
weighted avg       0.87      0.87      0.87      6513



In [11]:
# 로지스틱 회귀 모델 학습
lr_model = LogisticRegression(max_iter=1000)  # max_iter는 수렴까지의 반복 횟수, 필요에 따라 조정 가능
lr_model.fit(X_train_scaled, y_train)

# 예측
y_pred = lr_model.predict(X_test_scaled)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8278826961461692
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.94      0.89      4945
           1       0.72      0.46      0.56      1568

    accuracy                           0.83      6513
   macro avg       0.79      0.70      0.73      6513
weighted avg       0.82      0.83      0.81      6513



In [12]:
# Decision Tree 모델 학습
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# 예측
y_pred = dt_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8129894058037771
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.87      0.88      4945
           1       0.61      0.64      0.62      1568

    accuracy                           0.81      6513
   macro avg       0.74      0.75      0.75      6513
weighted avg       0.82      0.81      0.81      6513

